In [1]:
import itertools

import pandas as pd
import gffutils
import pybedtools
import re
import numpy as np

v19db_filename = '/projects/ps-yeolab/genomes/hg19/gencode/v19/gencode.v19.annotation.gtf.db'
v19db = gffutils.FeatureDB(v19db_filename)

folder = '/projects/ps-yeolab/obotvinnik/singlecell_pnms'

se_df = pd.read_csv('{}/exons_to_junctions_se.csv'.format(folder), index_col=0)
se_df.head()

/home/obotvinnik/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:784: UserWarning: Found matplotlib configuration in ~/.matplotlib/. To conform with the XDG base directory standard, this configuration location has been deprecated on Linux, and the new location is now /home/obotvinnik/.config/matplotlib/. Please move your configuration there to ensure that matplotlib will continue to find it in the future.
  _get_xdg_config_dir())


,exon1,exon2,exon3,junction_exon12,junction_exon23,junction_exon13,event_id
0,exon:chr10:100190328-100190427:-,exon:chr10:100189548-100189646:-,exon:chr10:100188908-100189399:-,chr10:100189647-100190327:-,chr10:100189400-100189547:-,chr10:100189400-100190327:-,exon:chr10:100190328-100190427:-@exon:chr10:10...
1,exon:chr10:100190328-100190427:-,exon:chr10:100189548-100189646:-,exon:chr10:100188913-100189399:-,chr10:100189647-100190327:-,chr10:100189400-100189547:-,chr10:100189400-100190327:-,exon:chr10:100190328-100190427:-@exon:chr10:10...
2,exon:chr10:100190328-100190427:-,exon:chr10:100189548-100189646:-,exon:chr10:100189330-100189399:-,chr10:100189647-100190327:-,chr10:100189400-100189547:-,chr10:100189400-100190327:-,exon:chr10:100190328-100190427:-@exon:chr10:10...
3,exon:chr10:100193697-100193848:-,exon:chr10:100190888-100191048:-,exon:chr10:100190328-100190427:-,chr10:100191049-100193696:-,chr10:100190428-100190887:-,chr10:100190428-100193696:-,exon:chr10:100193697-100193848:-@exon:chr10:10...
4,exon:chr10:100195392-100195529:-,exon:chr10:100195029-100195171:-,exon:chr10:100193697-100193848:-,chr10:100195172-100195391:-,chr10:100193849-100195028:-,chr10:100193849-100195391:-,exon:chr10:100195392-100195529:-@exon:chr10:10...


In [5]:
exon_cols = ['exon1', 'exon2', 'exon3']

In [3]:
mxe_df = pd.read_csv('/projects/ps-yeolab/obotvinnik/singlecell_pnms/exons_to_junctions_mxe.csv', index_col=0)
mxe_df.head()

,exon1,exon2,exon3,exon4,junction13,junction34,junction12,junction24,event_id
0,exon:chr10:102003455-102003529:-,exon:chr10:101997779-101997988:-,exon:chr10:101996607-101996726:-,exon:chr10:101995424-101995521:-,chr10:101996727-102003454:-,chr10:101995522-101996606:-,chr10:101997989-102003454:-,chr10:101995522-101997778:-,exon:chr10:102003455-102003529:-@exon:chr10:10...
1,exon:chr10:102003455-102003534:-,exon:chr10:101997779-101997988:-,exon:chr10:101996607-101996726:-,exon:chr10:101995424-101995521:-,chr10:101996727-102003454:-,chr10:101995522-101996606:-,chr10:101997989-102003454:-,chr10:101995522-101997778:-,exon:chr10:102003455-102003534:-@exon:chr10:10...
2,exon:chr10:102013178-102013296:-,exon:chr10:102010005-102010089:-,exon:chr10:102003455-102003534:-,exon:chr10:101997779-101997988:-,chr10:102003535-102013177:-,chr10:101997989-102003454:-,chr10:102010090-102013177:-,chr10:101997989-102010004:-,exon:chr10:102013178-102013296:-@exon:chr10:10...
3,exon:chr10:102013178-102014376:-,exon:chr10:102010005-102010089:-,exon:chr10:102003455-102003534:-,exon:chr10:101997779-101997988:-,chr10:102003535-102013177:-,chr10:101997989-102003454:-,chr10:102010090-102013177:-,chr10:101997989-102010004:-,exon:chr10:102013178-102014376:-@exon:chr10:10...
4,exon:chr10:102020723-102020785:-,exon:chr10:102019975-102020076:-,exon:chr10:102016019-102016233:-,exon:chr10:102013178-102013296:-,chr10:102016234-102020722:-,chr10:102013297-102016018:-,chr10:102020077-102020722:-,chr10:102013297-102019974:-,exon:chr10:102020723-102020785:-@exon:chr10:10...


In [5]:
from poshsplice.region import Region

def get_gene_attributes(df, exon_cols, db):
    df['gencode_id'] = df[exon_cols].apply(
        lambda x: ','.join(set(itertools.chain(*[db[i].attributes['gene_id'] for i in x]))), axis=1)
    df['gene_name'] = df[exon_cols].apply(
        lambda x: ','.join(set(itertools.chain(*[db[i].attributes['gene_name'] for i in x]))), axis=1)
    df['ensembl_id'] = df[exon_cols].apply(
        lambda x: ','.join(set(itertools.chain(*[map(lambda y: y.split('.')[0], db[i].attributes['gene_id']) for i in x]))), 
        axis=1)
    
    for exon_col in exon_cols:
        df['{}_region'.format(exon_col)] = df[exon_col].map(Region)
        df['{}_length'.format(exon_col)] = df['{}_region'.format(exon_col)].map(len)
    df['strand'] = df[exon_cols[0]].str[-1]
    return df

In [6]:
mxe_exon_cols = ['exon{}'.format(i) for i in range(1, 5)]
mxe_df_annotated = get_gene_attributes(mxe_df, mxe_exon_cols, v19db)

In [10]:
se_exon_cols = ['exon{}'.format(i) for i in range(1, 4)]
se_df_annotated = get_gene_attributes(se_df, se_exon_cols, v19db)

In [9]:
mxe_df_annotated.loc[mxe_df_annotated.gene_name == 'PKM']

,exon1,exon2,exon3,exon4,junction13,junction34,junction12,junction24,event_id,gencode_id,gene_name,ensembl_id,exon1_region,exon1_length,exon2_region,exon2_length,exon3_region,exon3_length,exon4_region,exon4_length
5099,exon:chr15:72499069-72499139:-,exon:chr15:72495363-72495529:-,exon:chr15:72494795-72494961:-,exon:chr15:72492815-72492996:-,chr15:72494962-72499068:-,chr15:72492997-72494794:-,chr15:72495530-72499068:-,chr15:72492997-72495362:-,exon:chr15:72499069-72499139:-@exon:chr15:7249...,ENSG00000067225.13,PKM,ENSG00000067225,exon:chr15:72499069-72499139:-,71,exon:chr15:72495363-72495529:-,167,exon:chr15:72494795-72494961:-,167,exon:chr15:72492815-72492996:-,182
5100,exon:chr15:72499069-72499195:-,exon:chr15:72495363-72495529:-,exon:chr15:72494795-72494961:-,exon:chr15:72492815-72492996:-,chr15:72494962-72499068:-,chr15:72492997-72494794:-,chr15:72495530-72499068:-,chr15:72492997-72495362:-,exon:chr15:72499069-72499195:-@exon:chr15:7249...,ENSG00000067225.13,PKM,ENSG00000067225,exon:chr15:72499069-72499195:-,127,exon:chr15:72495363-72495529:-,167,exon:chr15:72494795-72494961:-,167,exon:chr15:72492815-72492996:-,182
5101,exon:chr15:72499069-72499221:-,exon:chr15:72495363-72495529:-,exon:chr15:72494795-72494961:-,exon:chr15:72492815-72492996:-,chr15:72494962-72499068:-,chr15:72492997-72494794:-,chr15:72495530-72499068:-,chr15:72492997-72495362:-,exon:chr15:72499069-72499221:-@exon:chr15:7249...,ENSG00000067225.13,PKM,ENSG00000067225,exon:chr15:72499069-72499221:-,153,exon:chr15:72495363-72495529:-,167,exon:chr15:72494795-72494961:-,167,exon:chr15:72492815-72492996:-,182


In [19]:
positive_strand = exons_to_junctions.strand == '+'
exons_to_junctions['intron1_length'] = np.nan


exons_to_junctions.loc[positive_strand, 'intron1_length'] = \
    exons_to_junctions.loc[positive_strand].apply(lambda x: x.exon2_region._start - x.exon1_region._stop - 1, axis=1)
exons_to_junctions.loc[~positive_strand, 'intron1_length'] = \
    exons_to_junctions.loc[~positive_strand].apply(lambda x: x.exon1_region._start - x.exon2_region._stop - 1, axis=1)
exons_to_junctions.head()

,exon1,exon2,exon3,junctions,event_id,gencode_id,gene_name,ensembl_id,exon2_length,exon1_region,exon1_length,exon2_region,exon3_region,exon3_length,strand,intron1_length
0,exon:chr10:100190328-100190427:-,exon:chr10:100189548-100189646:-,exon:chr10:100188908-100189399:-,"(chr10:100189647-100190327:-, chr10:100189400-...",exon:chr10:100190328-100190427:-@exon:chr10:10...,ENSG00000107521.14,HPS1,ENSG00000107521,99,exon:chr10:100190328-100190427:-,100,exon:chr10:100189548-100189646:-,exon:chr10:100188908-100189399:-,492,-,681
1,exon:chr10:100190328-100190427:-,exon:chr10:100189548-100189646:-,exon:chr10:100188913-100189399:-,"(chr10:100189647-100190327:-, chr10:100189400-...",exon:chr10:100190328-100190427:-@exon:chr10:10...,ENSG00000107521.14,HPS1,ENSG00000107521,99,exon:chr10:100190328-100190427:-,100,exon:chr10:100189548-100189646:-,exon:chr10:100188913-100189399:-,487,-,681
2,exon:chr10:100190328-100190427:-,exon:chr10:100189548-100189646:-,exon:chr10:100189330-100189399:-,"(chr10:100189647-100190327:-, chr10:100189400-...",exon:chr10:100190328-100190427:-@exon:chr10:10...,ENSG00000107521.14,HPS1,ENSG00000107521,99,exon:chr10:100190328-100190427:-,100,exon:chr10:100189548-100189646:-,exon:chr10:100189330-100189399:-,70,-,681
3,exon:chr10:100193697-100193848:-,exon:chr10:100190888-100191048:-,exon:chr10:100190328-100190427:-,"(chr10:100191049-100193696:-, chr10:100190428-...",exon:chr10:100193697-100193848:-@exon:chr10:10...,ENSG00000107521.14,HPS1,ENSG00000107521,161,exon:chr10:100193697-100193848:-,152,exon:chr10:100190888-100191048:-,exon:chr10:100190328-100190427:-,100,-,2648
4,exon:chr10:100195392-100195529:-,exon:chr10:100195029-100195171:-,exon:chr10:100193697-100193848:-,"(chr10:100195172-100195391:-, chr10:100193849-...",exon:chr10:100195392-100195529:-@exon:chr10:10...,ENSG00000107521.14,HPS1,ENSG00000107521,143,exon:chr10:100195392-100195529:-,138,exon:chr10:100195029-100195171:-,exon:chr10:100193697-100193848:-,152,-,220


In [20]:
positive_strand = exons_to_junctions.strand == '+'
exons_to_junctions['intron2_length'] = np.nan


exons_to_junctions.loc[positive_strand, 'intron2_length'] = \
    exons_to_junctions.loc[positive_strand].apply(lambda x: x.exon3_region._start - x.exon2_region._stop - 1, axis=1)
exons_to_junctions.loc[~positive_strand, 'intron2_length'] = \
    exons_to_junctions.loc[~positive_strand].apply(lambda x: x.exon2_region._start - x.exon3_region._stop - 1, axis=1)
exons_to_junctions.head()

,exon1,exon2,exon3,junctions,event_id,gencode_id,gene_name,ensembl_id,exon2_length,exon1_region,exon1_length,exon2_region,exon3_region,exon3_length,strand,intron1_length,intron2_length
0,exon:chr10:100190328-100190427:-,exon:chr10:100189548-100189646:-,exon:chr10:100188908-100189399:-,"(chr10:100189647-100190327:-, chr10:100189400-...",exon:chr10:100190328-100190427:-@exon:chr10:10...,ENSG00000107521.14,HPS1,ENSG00000107521,99,exon:chr10:100190328-100190427:-,100,exon:chr10:100189548-100189646:-,exon:chr10:100188908-100189399:-,492,-,681,148
1,exon:chr10:100190328-100190427:-,exon:chr10:100189548-100189646:-,exon:chr10:100188913-100189399:-,"(chr10:100189647-100190327:-, chr10:100189400-...",exon:chr10:100190328-100190427:-@exon:chr10:10...,ENSG00000107521.14,HPS1,ENSG00000107521,99,exon:chr10:100190328-100190427:-,100,exon:chr10:100189548-100189646:-,exon:chr10:100188913-100189399:-,487,-,681,148
2,exon:chr10:100190328-100190427:-,exon:chr10:100189548-100189646:-,exon:chr10:100189330-100189399:-,"(chr10:100189647-100190327:-, chr10:100189400-...",exon:chr10:100190328-100190427:-@exon:chr10:10...,ENSG00000107521.14,HPS1,ENSG00000107521,99,exon:chr10:100190328-100190427:-,100,exon:chr10:100189548-100189646:-,exon:chr10:100189330-100189399:-,70,-,681,148
3,exon:chr10:100193697-100193848:-,exon:chr10:100190888-100191048:-,exon:chr10:100190328-100190427:-,"(chr10:100191049-100193696:-, chr10:100190428-...",exon:chr10:100193697-100193848:-@exon:chr10:10...,ENSG00000107521.14,HPS1,ENSG00000107521,161,exon:chr10:100193697-100193848:-,152,exon:chr10:100190888-100191048:-,exon:chr10:100190328-100190427:-,100,-,2648,460
4,exon:chr10:100195392-100195529:-,exon:chr10:100195029-100195171:-,exon:chr10:100193697-100193848:-,"(chr10:100195172-100195391:-, chr10:100193849-...",exon:chr10:100195392-100195529:-@exon:chr10:10...,ENSG00000107521.14,HPS1,ENSG00000107521,143,exon:chr10:100195392-100195529:-,138,exon:chr10:100195029-100195171:-,exon:chr10:100193697-100193848:-,152,-,220,1180


In [21]:
exons_to_junctions['intron12_length_summed'] = exons_to_junctions.intron1_length + exons_to_junctions.intron2_length
exons_to_junctions.head()

,exon1,exon2,exon3,junctions,event_id,gencode_id,gene_name,ensembl_id,exon2_length,exon1_region,exon1_length,exon2_region,exon3_region,exon3_length,strand,intron1_length,intron2_length,intron12_length_summed
0,exon:chr10:100190328-100190427:-,exon:chr10:100189548-100189646:-,exon:chr10:100188908-100189399:-,"(chr10:100189647-100190327:-, chr10:100189400-...",exon:chr10:100190328-100190427:-@exon:chr10:10...,ENSG00000107521.14,HPS1,ENSG00000107521,99,exon:chr10:100190328-100190427:-,100,exon:chr10:100189548-100189646:-,exon:chr10:100188908-100189399:-,492,-,681,148,829
1,exon:chr10:100190328-100190427:-,exon:chr10:100189548-100189646:-,exon:chr10:100188913-100189399:-,"(chr10:100189647-100190327:-, chr10:100189400-...",exon:chr10:100190328-100190427:-@exon:chr10:10...,ENSG00000107521.14,HPS1,ENSG00000107521,99,exon:chr10:100190328-100190427:-,100,exon:chr10:100189548-100189646:-,exon:chr10:100188913-100189399:-,487,-,681,148,829
2,exon:chr10:100190328-100190427:-,exon:chr10:100189548-100189646:-,exon:chr10:100189330-100189399:-,"(chr10:100189647-100190327:-, chr10:100189400-...",exon:chr10:100190328-100190427:-@exon:chr10:10...,ENSG00000107521.14,HPS1,ENSG00000107521,99,exon:chr10:100190328-100190427:-,100,exon:chr10:100189548-100189646:-,exon:chr10:100189330-100189399:-,70,-,681,148,829
3,exon:chr10:100193697-100193848:-,exon:chr10:100190888-100191048:-,exon:chr10:100190328-100190427:-,"(chr10:100191049-100193696:-, chr10:100190428-...",exon:chr10:100193697-100193848:-@exon:chr10:10...,ENSG00000107521.14,HPS1,ENSG00000107521,161,exon:chr10:100193697-100193848:-,152,exon:chr10:100190888-100191048:-,exon:chr10:100190328-100190427:-,100,-,2648,460,3108
4,exon:chr10:100195392-100195529:-,exon:chr10:100195029-100195171:-,exon:chr10:100193697-100193848:-,"(chr10:100195172-100195391:-, chr10:100193849-...",exon:chr10:100195392-100195529:-@exon:chr10:10...,ENSG00000107521.14,HPS1,ENSG00000107521,143,exon:chr10:100195392-100195529:-,138,exon:chr10:100195029-100195171:-,exon:chr10:100193697-100193848:-,152,-,220,1180,1400


In [31]:
exons_to_junctions.dtypes

exon1                      object
exon2                      object
exon3                      object
junctions                  object
event_id                   object
gencode_id                 object
gene_name                  object
ensembl_id                 object
exon2_length                int64
exon1_region               object
exon1_length                int64
exon2_region               object
exon3_region               object
exon3_length                int64
strand                     object
intron1_length            float64
intron2_length            float64
intron12_length_summed    float64
exon2_divisible_by_3         bool
dtype: object

In [32]:
exons_to_junctions['exon2_divisible_by_3'] = exons_to_junctions['exon2_length'].astype(int).mod(3) == 0
exons_to_junctions.head()

,exon1,exon2,exon3,junctions,event_id,gencode_id,gene_name,ensembl_id,exon2_length,exon1_region,exon1_length,exon2_region,exon3_region,exon3_length,strand,intron1_length,intron2_length,intron12_length_summed,exon2_divisible_by_3
0,exon:chr10:100190328-100190427:-,exon:chr10:100189548-100189646:-,exon:chr10:100188908-100189399:-,"(chr10:100189647-100190327:-, chr10:100189400-...",exon:chr10:100190328-100190427:-@exon:chr10:10...,ENSG00000107521.14,HPS1,ENSG00000107521,99,exon:chr10:100190328-100190427:-,100,exon:chr10:100189548-100189646:-,exon:chr10:100188908-100189399:-,492,-,681,148,829,True
1,exon:chr10:100190328-100190427:-,exon:chr10:100189548-100189646:-,exon:chr10:100188913-100189399:-,"(chr10:100189647-100190327:-, chr10:100189400-...",exon:chr10:100190328-100190427:-@exon:chr10:10...,ENSG00000107521.14,HPS1,ENSG00000107521,99,exon:chr10:100190328-100190427:-,100,exon:chr10:100189548-100189646:-,exon:chr10:100188913-100189399:-,487,-,681,148,829,True
2,exon:chr10:100190328-100190427:-,exon:chr10:100189548-100189646:-,exon:chr10:100189330-100189399:-,"(chr10:100189647-100190327:-, chr10:100189400-...",exon:chr10:100190328-100190427:-@exon:chr10:10...,ENSG00000107521.14,HPS1,ENSG00000107521,99,exon:chr10:100190328-100190427:-,100,exon:chr10:100189548-100189646:-,exon:chr10:100189330-100189399:-,70,-,681,148,829,True
3,exon:chr10:100193697-100193848:-,exon:chr10:100190888-100191048:-,exon:chr10:100190328-100190427:-,"(chr10:100191049-100193696:-, chr10:100190428-...",exon:chr10:100193697-100193848:-@exon:chr10:10...,ENSG00000107521.14,HPS1,ENSG00000107521,161,exon:chr10:100193697-100193848:-,152,exon:chr10:100190888-100191048:-,exon:chr10:100190328-100190427:-,100,-,2648,460,3108,False
4,exon:chr10:100195392-100195529:-,exon:chr10:100195029-100195171:-,exon:chr10:100193697-100193848:-,"(chr10:100195172-100195391:-, chr10:100193849-...",exon:chr10:100195392-100195529:-@exon:chr10:10...,ENSG00000107521.14,HPS1,ENSG00000107521,143,exon:chr10:100195392-100195529:-,138,exon:chr10:100195029-100195171:-,exon:chr10:100193697-100193848:-,152,-,220,1180,1400,False


In [33]:
exons_to_junctions['exon2_divisible_by_3'].sum()/float(exons_to_junctions.shape[0])

0.38737636382175999

In [34]:
cols = ['event_id', 'gencode_id', 'ensembl_id', 'gene_name', 'exon1_length', 
        'exon2_length', 'exon3_length', 'intron1_length', 
        'intron2_length', 'exon2_divisible_by_3', 'intron12_length_summed']
exon_metadata = exons_to_junctions[cols]
exon_metadata = exon_metadata.set_index('event_id')
exon_metadata.head()

,gencode_id,ensembl_id,gene_name,exon1_length,exon2_length,exon3_length,intron1_length,intron2_length,exon2_divisible_by_3,intron12_length_summed
event_id,,,,,,,,,,
exon:chr10:100190328-100190427:-@exon:chr10:100189548-100189646:-@exon:chr10:100188908-100189399:-,ENSG00000107521.14,ENSG00000107521,HPS1,100,99,492,681,148,True,829
exon:chr10:100190328-100190427:-@exon:chr10:100189548-100189646:-@exon:chr10:100188913-100189399:-,ENSG00000107521.14,ENSG00000107521,HPS1,100,99,487,681,148,True,829
exon:chr10:100190328-100190427:-@exon:chr10:100189548-100189646:-@exon:chr10:100189330-100189399:-,ENSG00000107521.14,ENSG00000107521,HPS1,100,99,70,681,148,True,829
exon:chr10:100193697-100193848:-@exon:chr10:100190888-100191048:-@exon:chr10:100190328-100190427:-,ENSG00000107521.14,ENSG00000107521,HPS1,152,161,100,2648,460,False,3108
exon:chr10:100195392-100195529:-@exon:chr10:100195029-100195171:-@exon:chr10:100193697-100193848:-,ENSG00000107521.14,ENSG00000107521,HPS1,138,143,152,220,1180,False,1400


In [35]:
print exon_metadata.ensembl_id.str.contains(',').sum()
print exon_metadata.shape

3928
(58842, 10)


3,928/58,842 splicing events have multiple gene ids associated with them. Since we want to intersect this information with existing gene annotations, we need a *single* id. To do this coarsely and simply, we will take the first gene ID for intersecting purposes.

In [36]:
exon_metadata['one_ensembl_id'] = exon_metadata.ensembl_id.str.split(',').str[0]
exon_metadata.head()

,gencode_id,ensembl_id,gene_name,exon1_length,exon2_length,exon3_length,intron1_length,intron2_length,exon2_divisible_by_3,intron12_length_summed,one_ensembl_id
event_id,,,,,,,,,,,
exon:chr10:100190328-100190427:-@exon:chr10:100189548-100189646:-@exon:chr10:100188908-100189399:-,ENSG00000107521.14,ENSG00000107521,HPS1,100,99,492,681,148,True,829,ENSG00000107521
exon:chr10:100190328-100190427:-@exon:chr10:100189548-100189646:-@exon:chr10:100188913-100189399:-,ENSG00000107521.14,ENSG00000107521,HPS1,100,99,487,681,148,True,829,ENSG00000107521
exon:chr10:100190328-100190427:-@exon:chr10:100189548-100189646:-@exon:chr10:100189330-100189399:-,ENSG00000107521.14,ENSG00000107521,HPS1,100,99,70,681,148,True,829,ENSG00000107521
exon:chr10:100193697-100193848:-@exon:chr10:100190888-100191048:-@exon:chr10:100190328-100190427:-,ENSG00000107521.14,ENSG00000107521,HPS1,152,161,100,2648,460,False,3108,ENSG00000107521
exon:chr10:100195392-100195529:-@exon:chr10:100195029-100195171:-@exon:chr10:100193697-100193848:-,ENSG00000107521.14,ENSG00000107521,HPS1,138,143,152,220,1180,False,1400,ENSG00000107521


### Write to a safe place

In [37]:
exon_metadata.to_csv('{}/splicing_feature_data.csv'.format(folder))